In [17]:
import numpy as np
import math
import matplotlib.pyplot as plt
import shapely
from shapely import Point
from shapely import box
from itertools import combinations

Ser=    [2000,   0,   0,   2500,   1200]                      # Ps, Vsx, Vsy, Msx, Msy
Uls=    [1200,   0,   0,   2000,   1600]                      # Pu, Vux, Vuy, Mux, Muy
Pile=   [1300,  300,     200,     0.5,     3,     0.1]        # P, V, T Capacity of Pile, Diameter of Pile, Pile Spacing, Embed depth
Mat=    [24,    400,     0.05]                                # f'c, fy, cc
SpW=    [0,     0,       0,    1.5]                           # Surface, Conc, Soil, height
Ped=    [0.5,   0.8]                                          # Pedestal Size

PileCap= [0.55,     9]                                        # Overall Depth, Pile Q'ty
Rebar=   [22,  15,  150, 150, 150, 150]                       # Bottom Rebar Dia, Top Rebar Dia, Bottom X Spacing, Bottom Y Spacing, Top X Spacing, Top Y Spacing

S=      Pile[3]*Pile[4]
Pi = 0.75
λ = 1
vc = 0.33 * λ * math.sqrt(Mat[0]) * 1000                                    # 2 way shear 기준 vc (1 way는 이 값의 절반 사용)
Cx = Ped[0]
Cy = Ped[1]

In [18]:
Xi=     np.array([
        [0, 0,    0, -S/2, -S*3/4, -S/2,       0, -S*9/10, -S,       0],
        [0, 0, -S/2,  S/2,  S*3/4,  S/2, -S*9/10,  S*9/10,  0, -S*9/10],
        [0, 0,  S/2, -S/2,      0, -S/2,  S*9/10,       0,  S,  S*9/10],
        [0, 0,    0,  S/2, -S*3/4,  S/2,       0, -S*9/10, -S,       0],
        [0, 0,    0,    0,  S*3/4, -S/2, -S*9/10,  S*9/10,  0, -S*9/10],
        [0, 0,    0,    0,      0,  S/2,  S*9/10,       0,  S,  S*9/10],
        [0, 0,    0,    0,      0,    0,       0, -S*9/10, -S,       0],
        [0, 0,    0,    0,      0,    0,       0,  S*9/10,  0, -S*9/10],
        [0, 0,    0,    0,      0,    0,       0,       0,  S,  S*9/10],
        [0, 0,    0,    0,      0,    0,       0,       0,  0,       0],
                 ])


Yi=     np.array([
        [0,  S/2,   S*3/5,  S/2,  S*3/4,  S,    S,    S,  S,  S*3/2],
        [0, -S/2, -S*3/10,  S/2,  S*3/4,  S,  S/2,    S,  S,      S],
        [0,    0, -S*3/10, -S/2,      0,  0,  S/2,  S/2,  S,      S],
        [0,    0,       0, -S/2, -S*3/4,  0,    0,    0,  0,    S/2],
        [0,    0,       0,    0, -S*3/4, -S, -S/2,    0,  0,      0],
        [0,    0,       0,    0,      0, -S, -S/2, -S/2,  0,      0],
        [0,    0,       0,    0,      0,  0,   -S,   -S, -S,   -S/2],
        [0,    0,       0,    0,      0,  0,    0,   -S, -S,     -S],
        [0,    0,       0,    0,      0,  0,    0,    0, -S,     -S],
        [0,    0,       0,    0,      0,  0,    0,    0,  0, -S*3/2],
                ])

Lxy=    np.array([
         [S,   S,     2*S, 2*S, S*5/2, 2*S, 28*S/10, 28*S/10, 3*S, 28*S/10],
         [S, 2*S, 19*S/10, 2*S, S*5/2, 3*S,     3*S,     3*S, 3*S,     4*S]
                ])

Xi2=    Xi**2
Yi2=    Yi**2

In [19]:
def Ceil(x):
    return (x*20-x*20%1+1)/20                                                                                # 0.05 올림 식 정의
def Two_Way_Shear_From_Column_Face_Capa(PileCapDepth, Cx, Cy):
    return Pi*vc*2*(Cx+Cy+2*PileCapDepth)*PileCapDepth                                                       # 2 Way Shear at d/2 from Column face 식 정의
def Two_Way_Shear_From_Column_Face_Special_Capa(PileCapDepth, Cx, Cy):
    return Pi*vc*2*(Cx+Cy)*PileCapDepth                                                                      # 2 Way Shear at d/2 from Column face 식 정의 (Special Provision)
def One_Way_Shear_From_Column_Face_Capa(PileCapDepth,PileCapLength):
    return Pi*vc/2*PileCapDepth*PileCapLength                                                                # 1 Way Shear at d from Column face 식 정의
def Two_Way_Shear_From_Pile_Capa(PileCapDepth, Pile):
    return Pi*vc*math.pi*(Pile[3]+PileCapDepth)*PileCapDepth                                                 # 2 Way Shear at d/2 from Pile (Internal) 식 정의 1
def Two_Way_Shear_From_Pile_Group_Capa(PileCapDepth, Pile, S):
    return Pi*vc*(math.pi*(Pile[3]+PileCapDepth)+2*S)*PileCapDepth                                           # 2 Way Shear at d/2 from Pile (Internal) 식 정의 2
def Two_Way_Shear_At_Corner_Pile_Edge_Capa(PileCapDepth, Pile, S):
    return Pi*vc*(math.pi*(Pile[3]+PileCapDepth)/4+math.sqrt(2)*S+(Pile[3]+PileCapDepth))*PileCapDepth       # 2 Way Shear at Corner Pile 식 정의 1
def Two_Way_Shear_At_Corner_Pile_Corner_Capa(PileCapDepth, Pile, S):
    return Pi*vc*(math.pi*(Pile[3]+PileCapDepth)/4+S)*PileCapDepth                                           # 2 Way Shear at Corner Pile 식 정의 2
def One_Way_Shear_At_Corner_Pile_Capa(PileCapDepth, Pile, S):
    CriticalDistance = min(PileCapDepth, 0.3)
    return Pi*vc/2*(math.sqrt(2)*S+(Pile[3]+2*CriticalDistance))*PileCapDepth                                      # 1 Way Shear at Corner Pile 식 정의

nlat=   math.ceil(max(Ser[1],Ser[2])/Pile[1])
nver=   math.ceil(Ser[0]/Pile[0])

In [20]:
dt = PileCap[0]-Pile[5]-Mat[2]                  # Pile Cap Clear Depth
n = PileCap[1]                                  # Pile Qty

def Distributed_Load_on_Pile_Cap(dt, Pile, Mat, SpW) : 
        w = SpW[0]+(dt+Pile[5]+Mat[2])*SpW[1]+(SpW[3]-(dt+Pile[5]+Mat[2]))*SpW[2]
        return w

def Pile_Service_Axial_Load(w, Ser, Lxy, n) : 
        Pxser = [
                (Ser[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Ser[4])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Ser[3])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
                for i in range(n)
                ]
        Pyser = [
                (Ser[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Ser[3])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Ser[4])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
                for i in range(n)
                ]
        return Pxser, Pyser

def Define_Pile_Arrangement_Direction(Pxser, Pyser, Pile) :
        if np.max(Pxser)<Pile[0] and (np.min(Pxser)>0 or abs(np.min(Pxser))<Pile[2]):                           # X 방향 배치가 OK 일 시
                dir = 1
        elif np.max(Pyser)<Pile[0] and (np.min(Pyser)>0 or abs(np.min(Pyser))<Pile[2]):                         # Y 방향 배치가 OK 일 시
                dir = 2
        else :
                dir = 1
        return dir

def Pile_Service_Capacity_Check(Pxser, Pyser, Ser, Pile, n) : 
        lat = max(Ser[1],Ser[2])/n
        if (np.max(Pxser)>Pile[0] and np.max(Pyser)>Pile[0]) or ((np.min(Pxser)<0 and abs(np.min(Pxser))>Pile[2]) and (np.min(Pyser)<0 and abs(np.min(Pyser))>Pile[2])) or lat > Pile[1] : 
                Capa = 'NG'
        else : 
                Capa = 'OK'
        return Capa

def Pile_Factored_Axial_Load(w, dir, Uls, Xi, Yi, Lxy, n) :
        Pxuls =    [
                (Uls[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Uls[4])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Uls[3])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
                for i in range(n)
                ]
        Pyuls =    [
                (Uls[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Uls[3])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Uls[4])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
                for i in range(n)
                ]
        if dir == 1:
                Puls = Pxuls               # 계수하중에 의한 Pile 반력 (X 방향 배치가 OK일 시)
        elif dir == 2:
                Puls = Pyuls               # 계수하중에 의한 Pile 반력 (Y 방향 배치가 OK일 시)
        return Puls

def Pile_Location(PileQty, Xi, Yi, Pile) :
        PileLocation = [
            Point(Xi[i,PileQty-1],Yi[i,PileQty-1]).buffer(Pile[3]/2, quad_segs=500)
            for i in range(PileQty)
            ]
        return PileLocation

w = Distributed_Load_on_Pile_Cap(dt, Pile, Mat, SpW)
Pxser, Pyser = Pile_Service_Axial_Load(w, Ser, Lxy, n)
dir = Define_Pile_Arrangement_Direction(Pxser, Pyser, Pile)
Capa = Pile_Service_Capacity_Check(Pxser, Pyser, Ser, Pile, n)
Puls = Pile_Factored_Axial_Load(w, dir, Uls, Xi, Yi, Lxy, n)
PileLocation = Pile_Location(n, Xi, Yi, Pile)

In [21]:
# 2 way shear at d/2 from column face

def Two_way_shear_from_column_face_Pile_Reaction(dt, Cx, Cy, Pile, Puls, PileLocation, n) :                                           
    d1 = dt
    TwoWayCritical = box(-(Cx/2+d1/2),-(Cy/2+d1/2),(Cx/2+d1/2),(Cy/2+d1/2))
    TwoWayReaction = [
                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],TwoWayCritical).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                    for i in range(n)
                    ]
    SP = 'NA'
    if TwoWayReaction == 0 :
        SpecialProvision = box(-(Cx/2),-(Cy/2),(Cx/2),(Cy/2))
        SpecialProvisionReaction = [
                                (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],SpecialProvision).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                                for i in range(n)
                                ]
        TwoWayReaction = SpecialProvisionReaction
        SP = 'A'
    return abs(sum(TwoWayReaction)), SP

TwoWayReaction1, SP = Two_way_shear_from_column_face_Pile_Reaction(dt, Cx, Cy, Pile, Puls, PileLocation, n)
Vu1 = TwoWayReaction1

In [22]:
# 1 way shear at d from column face

def One_way_shear_from_column_face_Pile_Reaction(dt, Puls, Cx, Cy, Xi, Yi, S, Pile, PileLocation, n) :
    d2 = dt
    OneWayCritical1 = box(min([Xi[i][n-1] for i in range(n)])-S/2,min([Yi[i][n-1] for i in range(n)])-S/2,max([Xi[i][n-1] for i in range(n)])+S/2,(Cy/2+d2))
    OneWayCritical2 = box(min([Xi[i][n-1] for i in range(n)])-S/2,min([Yi[i][n-1] for i in range(n)])-S/2,(Cx/2+d2),max([Yi[i][n-1] for i in range(n)])+S/2)
    OneWayCritical3 = box(min([Xi[i][n-1] for i in range(n)])-S/2,-(Cy/2+d2),max([Xi[i][n-1] for i in range(n)])+S/2,max([Yi[i][n-1] for i in range(n)])+S/2)
    OneWayCritical4 = box(-(Cx/2+d2),min([Yi[i][n-1] for i in range(n)])-S/2,max([Xi[i][n-1] for i in range(n)])+S/2,max([Yi[i][n-1] for i in range(n)])+S/2)
    OneWayReaction1 = [
                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],OneWayCritical1).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                    for i in range(n)
                    ]
    OneWayReaction2 = [
                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],OneWayCritical2).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                    for i in range(n)
                    ]
    OneWayReaction3 = [
                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],OneWayCritical3).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                    for i in range(n)
                    ]
    OneWayReaction4 = [
                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],OneWayCritical4).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                    for i in range(n)
                    ]
    if round(sum(OneWayReaction1),3) == 0 and round(sum(OneWayReaction2),3) == 0 and round(sum(OneWayReaction3),3) == 0 and round(sum(OneWayReaction4),3) == 0 :
        SpecialProvision1 = box(min([Xi[i][n-1] for i in range(n)])-S/2,min([Yi[i][n-1] for i in range(n)])-S/2,max([Xi[i][n-1] for i in range(n)])+S/2,(Cy/2))
        SpecialProvision2 = box(min([Xi[i][n-1] for i in range(n)])-S/2,min([Yi[i][n-1] for i in range(n)])-S/2,(Cx/2),max([Yi[i][n-1] for i in range(n)])+S/2)
        SpecialProvision3 = box(min([Xi[i][n-1] for i in range(n)])-S/2,-(Cy/2),max([Xi[i][n-1] for i in range(n)])+S/2,max([Yi[i][n-1] for i in range(n)])+S/2)
        SpecialProvision4 = box(-(Cx/2),min([Yi[i][n-1] for i in range(n)])-S/2,max([Xi[i][n-1] for i in range(n)])+S/2,max([Yi[i][n-1] for i in range(n)])+S/2)
        SpecialProvisionReaction1 = [
                                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],SpecialProvision1).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                                    for i in range(n)
                                    ]
        SpecialProvisionReaction2 = [
                                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],SpecialProvision2).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                                    for i in range(n)
                                    ]
        SpecialProvisionReaction3 = [
                                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],SpecialProvision3).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                                    for i in range(n)
                                    ]
        SpecialProvisionReaction4 = [
                                    (round(math.pi*(Pile[3]/2)**2,3)-round(shapely.intersection(PileLocation[i],SpecialProvision4).area,3))/round(math.pi*(Pile[3]/2)**2,3)*Puls[i]
                                    for i in range(n)
                                    ]
        OneWayReaction1 = SpecialProvisionReaction1
        OneWayReaction2 = SpecialProvisionReaction2
        OneWayReaction3 = SpecialProvisionReaction3
        OneWayReaction4 = SpecialProvisionReaction4
    return abs(max(abs(sum(OneWayReaction1)),abs(sum(OneWayReaction2)),abs(sum(OneWayReaction3)),abs(sum(OneWayReaction4)))), OneWayReaction1, OneWayReaction2, OneWayReaction3, OneWayReaction4

OneWayReaction1, OneWayReactionDC1, OneWayReactionDC2, OneWayReactionDC3, OneWayReactionDC4 = One_way_shear_from_column_face_Pile_Reaction(dt, Puls, Cx, Cy, Xi, Yi, S, Pile, PileLocation, n)
Vu2 = OneWayReaction1


In [23]:
# 2 Way Shear at d/2 from Pile (Internal)

def create_circle_dict(circles, attribute):
    return {'Poly': circles, 'Reaction': attribute}

def distance_between_circles(circle1, circle2):
    return round(circle1['Poly'].centroid.distance(circle2['Poly'].centroid),2)

def find_circle_combinations(circles, targetdistance):
    combination_list = []
    for circle1, circle2 in combinations(circles, 2):
        distance = distance_between_circles(circle1, circle2)
        if distance < targetdistance:
            combination_list.append((circle1, circle2))
    return combination_list

def calculate_attribute_sum(combination):
    return sum([circle['Reaction'] for circle in combination])

def Two_way_shear_from_pile_Pile_Reaction(PileGroup, dt, Puls, S, Pile) : 
    if dt <= (S-Pile[3])/2 :
        TwoWayReaction2 = max(list(map(abs,Puls)))
    else :
        attribute_sum = [
                        calculate_attribute_sum(combination) for combination in PileGroup
                        ]
        TwoWayReaction2 = abs(max(attribute_sum))
    return TwoWayReaction2

PileLandR = [create_circle_dict(PileLocation[i], Puls[i]) for i in range(n)]
PileGroup = find_circle_combinations(PileLandR, 1.1*S)
TwoWayReaction2 = Two_way_shear_from_pile_Pile_Reaction(PileGroup, dt, Puls, S, Pile)
Vu3 = TwoWayReaction2

In [24]:
# 2 Way Shear at Corner Pile 

def find_circles_distance_from_edge1(circles, Xi, Yi, S, n):
    res_edge1 = []
    for circle in circles:
        distance_x = abs(min(Xi[i][n-1] for i in range(n)))+S/2 - abs(circle['Poly'].centroid.x)
        distance_y = abs(min(Yi[i][n-1] for i in range(n)))+S/2 - abs(circle['Poly'].centroid.y)
        if (distance_x < 1.1*S/2) and (distance_y < 1.1*S/2) :
            res_edge1.append(circle)
    return res_edge1

def find_circles_distance_from_edge2(circles, Xi, Yi, S, n):
    res_edge2 = []
    for circle in circles:
        distance_x = abs(min(Xi[i][n-1] for i in range(n)))+S/2 - abs(circle['Poly'].centroid.x)
        distance_y = abs(min(Yi[i][n-1] for i in range(n)))+S/2 - abs(circle['Poly'].centroid.y)
        if ((distance_x < 1.1*S/2) and (distance_y > 1.1*S/2)) or ((distance_x > 1.1*S/2) and (distance_y < 1.1*S/2)) :
            res_edge2.append(circle)
    return res_edge2

def Reaction_Extract(circles):
    res_reaction = []
    for circle in circles:
        res_reaction.append(circle['Reaction'])
    return res_reaction

def max_in_list(lst):
    if lst:  
        return max(lst)
    else: 
        return 0
    
def Two_way_shear_corner_Pile_Reaction(PileLandR, Xi, Yi, S, n) : 
    if max_in_list(list(map(abs,Reaction_Extract(find_circles_distance_from_edge1(PileLandR, Xi, Yi, S, n))))) < max_in_list(list(map(abs,Reaction_Extract(find_circles_distance_from_edge2(PileLandR, Xi, Yi, S, n))))) : 
        TwoWayReaction3 = max_in_list(map(abs,Reaction_Extract(find_circles_distance_from_edge2(PileLandR, Xi, Yi, S, n))))
    else :
        TwoWayReaction3 = max_in_list(list(map(abs,Reaction_Extract(find_circles_distance_from_edge1(PileLandR, Xi, Yi, S, n)))))
    return TwoWayReaction3

TwoWayReaction3 = Two_way_shear_corner_Pile_Reaction(PileLandR, Xi, Yi, S, n)
Vu4 = TwoWayReaction3

In [25]:
# 1 Way Shear at Corner Pile

def One_way_shear_corner_Pile_Reaction(PileLandR, Xi, Yi, S, n) :
    OneWayReaction2 = max_in_list(map(abs,Reaction_Extract(find_circles_distance_from_edge1(PileLandR, Xi, Yi, S, n))))
    return OneWayReaction2

OneWayReaction2 = One_way_shear_corner_Pile_Reaction(PileLandR, Xi, Yi, S, n)
Vu5 = OneWayReaction2

In [26]:
# 2 way shear at d/2 from column face Vc 계산

def Two_way_shear_from_column_face_PileCap_Capa(PileCapDepth, Cx, Cy, SP) :
    if SP == 'NA' : 
        ΦVc1 = Two_Way_Shear_From_Column_Face_Capa(PileCapDepth, Cx, Cy)
    else :
        ΦVc1 = Two_Way_Shear_From_Column_Face_Special_Capa(PileCapDepth, Cx, Cy)
    return ΦVc1

ΦVc1 = Two_way_shear_from_column_face_PileCap_Capa(dt, Cx, Cy, SP)

In [27]:
# 1 way shear at d from column face Vc 계산

def One_way_shear_from_column_face_PileCap_Capa(PileCapDepth, Lxy, OneWayReactionDC1, OneWayReactionDC2, OneWayReactionDC3, OneWayReactionDC4, n) :
    if (sum(OneWayReactionDC1) or sum(OneWayReactionDC3)) == max(sum(OneWayReactionDC1),sum(OneWayReactionDC2),sum(OneWayReactionDC3),sum(OneWayReactionDC4)) :
        PileCapLength = Lxy[0][n-1]
        ΦVc2 = One_Way_Shear_From_Column_Face_Capa(PileCapDepth,PileCapLength)
    else : 
        PileCapLength = Lxy[1][n-1]
        ΦVc2 = One_Way_Shear_From_Column_Face_Capa(PileCapDepth,PileCapLength)
    return ΦVc2
    
ΦVc2 = One_way_shear_from_column_face_PileCap_Capa(dt, Lxy, OneWayReactionDC1, OneWayReactionDC2, OneWayReactionDC3, OneWayReactionDC4, n)

In [28]:
# 2 Way Shear at d/2 from Pile (Internal) Vc 계산

def Two_way_shear_from_pile_PileCap_Capa(PileCapDepth, Pile, S) :
    if PileCapDepth <= (S-Pile[3])/2 :
        ΦVc3 = Two_Way_Shear_From_Pile_Capa(PileCapDepth, Pile)
    else :
        ΦVc3 = Two_Way_Shear_From_Pile_Group_Capa(PileCapDepth, Pile, S)
    return ΦVc3

ΦVc3 = Two_way_shear_from_pile_PileCap_Capa(dt, Pile, S)

In [29]:
# 2 Way Shear at Corner Pile Vc 계산

def Two_way_shear_corner_PileCap_Capa(PileCapDepth, Pile, PileLandR, Xi, Yi, S, n) :
    if max_in_list(list(map(abs,Reaction_Extract(find_circles_distance_from_edge1(PileLandR, Xi, Yi, S, n))))) < max_in_list(list(map(abs,Reaction_Extract(find_circles_distance_from_edge2(PileLandR, Xi, Yi, S, n))))) : 
        ΦVc4 = Two_Way_Shear_At_Corner_Pile_Edge_Capa(PileCapDepth, Pile, S)
    else :
        ΦVc4 = Two_Way_Shear_At_Corner_Pile_Corner_Capa(PileCapDepth, Pile, S)
    return ΦVc4

ΦVc4 = Two_way_shear_corner_PileCap_Capa(dt, Pile, PileLandR, Xi, Yi, S, n)

In [30]:
# 1 Way Shear at Corner Pile Vc 계산

def One_way_shear_corner_PileCap_Capa(PileCapDepth, Pile, S) : 
    ΦVc5 = One_Way_Shear_At_Corner_Pile_Capa(PileCapDepth, Pile, S)
    return ΦVc5

ΦVc5 = One_way_shear_corner_PileCap_Capa(dt, Pile, S)

In [31]:
#### Output 확인용 ######## Output 확인용 ######## Output 확인용 ######## Output 확인용 ######## Output 확인용 ######## Output 확인용 ####

if dir == 1 :

        print("Pile n = " + str(n))
        print("Ps = " + str(Pile[0]))
        print("Pu = " + str(np.max(Pxser)))

        if np.min(Pxser)>0:
                print("Ts = " + str(Pile[2]) + " > 0")
        else :
                print("Ts = " + str(Pile[2]))
                print("Tu = " + str(abs(np.min(Pxser))))

        print("Pxser = " + str(Pxser))

elif dir == 2 :
        
        print("Pile n = " + str(n))
        print("Ps = " + str(Pile[0]))
        print("Pu = " + str(np.max(Pyser)))
        
        if np.min(Pyser)>0:
                print("Ts = " + str(Pile[2]) + " > 0")
        else :
                print("Ts = " + str(Pile[2]))
                print("Tu = " + str(abs(np.min(Pyser))))
                
        print("Pyser = " + str(Pyser))
else : 
        pass

print("Pile Capa = " + str(Capa))
print("Puls = " + str(Puls) + "\n\n")
print("d = " + str(round(dt,2)) + " (Governing Thickness)")
print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1,3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2,3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc5,3)) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")

if ΦVc1>Vu1 and ΦVc2>Vu2 and ΦVc3>Vu3 and ΦVc4>Vu4 and ΦVc5>Vu5 :
        print("Pile Cap Capa = " + "OK")
else : 
        print("Pile Cap Capa = " + "NG")

#### Output 확인용 ######## Output 확인용 ######## Output 확인용 ######## Output 확인용 ######## Output 확인용 ######## Output 확인용 ####

Pile n = 9
Ps = 1300
Pu = 633.3333333333333
Ts = 200
Tu = 188.88888888888889
Pxser = [366.66666666666663, 500.0, 633.3333333333333, 88.88888888888889, 222.22222222222223, 355.55555555555554, -188.88888888888889, -55.55555555555554, 77.77777777777777]
Pile Capa = OK
Puls = [177.7777777777778, 355.55555555555554, 533.3333333333333, -44.44444444444443, 133.33333333333334, 311.1111111111111, -266.66666666666663, -88.88888888888889, 88.88888888888886]


d = 0.4 (Governing Thickness)
2 Way Shear at d/2 From Column Face
    ΦVc = 2036.996 kN
    Vu = 1066.667 kN
1 Way Shear at d From Column Face
    ΦVc = 1091.248 kN
    Vu = 1066.667 kN
2 Way Shear at d/2 from Pile
    ΦVc = 1371.302 kN
    Vu = 533.333 kN
2 Way Shear at Corner Pile
    ΦVc = 1070.324 kN
    Vu = 533.333 kN
1 Way Shear at Corner Pile
    ΦVc = 781.169 kN
    Vu = 533.333 kN
Pile Cap Capa = OK


In [32]:
# Mu 계산

def find_circles_distance_from_axis(circles,location):
    res_x1 = []
    res_x2 = []
    res_y1 = []
    res_y2 = []
    if location == 'xplus' :
        for circle in circles:
            distance = circle['Poly'].centroid.x 
            if distance > 0.01 :
                res_x1.append(circle)
        return res_x1
    if location == 'xminus' :
        for circle in circles:
            distance = circle['Poly'].centroid.x 
            if distance < -0.01 :
                res_x2.append(circle)
        return res_x2
    if location == 'yplus' :
        for circle in circles:
            distance = circle['Poly'].centroid.y
            if distance > 0.01 :
                res_y1.append(circle)
        return res_y1
    if location == 'yminus' :
        for circle in circles:
            distance = circle['Poly'].centroid.y 
            if distance < -0.01 :
                res_y2.append(circle)
        return res_y2

def calculate_distance_attribute_product(circle,direction):
    if direction == 'xdir' :
        distance = abs(circle['Poly'].centroid.x)-Cx/2
        return distance * float(circle['Reaction'])
    if direction == 'ydir' :
        distance = abs(circle['Poly'].centroid.y)-Cy/2
        return distance * float(circle['Reaction'])

def Total_Moment(circles,direction) :
    res_mom = []
    for circle in circles :
        res_mom.append(calculate_distance_attribute_product(circle,direction))
    return sum(res_mom)

circles_within_Xdistance1 = find_circles_distance_from_axis(PileLandR,'xplus')
circles_within_Xdistance2 = find_circles_distance_from_axis(PileLandR,'xminus')
circles_within_Ydistance1 = find_circles_distance_from_axis(PileLandR,'yplus')
circles_within_Ydistance2 = find_circles_distance_from_axis(PileLandR,'yminus')

Muxbot = abs(max(Total_Moment(circles_within_Xdistance1,'xdir'),Total_Moment(circles_within_Xdistance2,'xdir'),0))
Muxtop = abs(min(Total_Moment(circles_within_Xdistance1,'xdir'),Total_Moment(circles_within_Xdistance2,'xdir'),0))
Muybot = abs(max(Total_Moment(circles_within_Ydistance1,'ydir'),Total_Moment(circles_within_Ydistance2,'ydir'),0))
Muytop = abs(min(Total_Moment(circles_within_Ydistance1,'ydir'),Total_Moment(circles_within_Ydistance2,'ydir'),0))

#각 방향별 휨 철근 설계

def Define_Effective_Depth (Rebar, Muxbot, Muxtop, Muybot, Muytop, dt) : 
    if Muxbot <= Muybot : 
        dxbot = (dt - Rebar[0]/(2*1000))*1000
        dybot = (dt - 3*Rebar[0]/(2*1000))*1000
    if Muxbot > Muybot : 
        dxbot = (dt - 3*Rebar[0]/(2*1000))*1000
        dybot = (dt - Rebar[0]/(2*1000))*1000
    if Muxtop <= Muytop : 
        dxtop = (dt - Rebar[1]/(2*1000))*1000
        dytop = (dt - 3*Rebar[1]/(2*1000))*1000
    if Muxtop > Muytop : 
        dxtop = (dt - 3*Rebar[1]/(2*1000))*1000
        dytop = (dt - Rebar[1]/(2*1000))*1000
    return dxbot, dybot, dxtop, dytop

def Calculate_Rebar_Force1(effdepth,Mu,Mat,Lxy) :
    return ((-0.9*0.85*Mat[0]*1000*effdepth)+((0.9*0.85*Mat[0]*1000*effdepth)**2-4*(-0.9*0.85*Mat[0]*1000*0.5)*(-Mu*10**6/Lxy[1][n-1]))**0.5)/(2*(-0.9*0.85*Mat[0]*1000*0.5))
def Calculate_Rebar_Force2(effdepth,Mu,Mat,Lxy) : 
    return ((-0.9*0.85*Mat[0]*1000*effdepth)-((0.9*0.85*Mat[0]*1000*effdepth)**2-4*(-0.9*0.85*Mat[0]*1000*0.5)*(-Mu*10**6/Lxy[0][n-1]))**0.5)/(2*(-0.9*0.85*Mat[0]*1000*0.5))

def Define_Applicable_Rebar_Force (dxbot, dybot, dxtop, dytop, Mat, Lxy) : 
    if Calculate_Rebar_Force1(dxbot,Muxbot,Mat,Lxy) < 0 :
        a = Calculate_Rebar_Force2(dxbot,Muxbot,Mat,Lxy)
    if Calculate_Rebar_Force2(dxbot,Muxbot,Mat,Lxy) < 0 : 
        a = Calculate_Rebar_Force1(dxbot,Muxbot,Mat,Lxy)
    if Calculate_Rebar_Force1(dxbot,Muxbot,Mat,Lxy) > dxbot : 
        a = Calculate_Rebar_Force2(dxbot,Muxbot,Mat,Lxy)
    if Calculate_Rebar_Force2(dxbot,Muxbot,Mat,Lxy) > dxbot : 
        a = Calculate_Rebar_Force1(dxbot,Muxbot,Mat,Lxy)
    if Calculate_Rebar_Force1(dybot,Muybot,Mat,Lxy) < 0 :
        b = Calculate_Rebar_Force2(dybot,Muybot,Mat,Lxy)
    if Calculate_Rebar_Force2(dybot,Muybot,Mat,Lxy) < 0 : 
        b = Calculate_Rebar_Force1(dybot,Muybot,Mat,Lxy)
    if Calculate_Rebar_Force1(dybot,Muybot,Mat,Lxy) > dybot : 
        b = Calculate_Rebar_Force2(dybot,Muybot,Mat,Lxy)
    if Calculate_Rebar_Force2(dybot,Muybot,Mat,Lxy) > dybot : 
        b = Calculate_Rebar_Force1(dybot,Muybot,Mat,Lxy)
    if Calculate_Rebar_Force1(dxtop,Muxtop,Mat,Lxy) < 0 :
        c = Calculate_Rebar_Force2(dxtop,Muxtop,Mat,Lxy)
    if Calculate_Rebar_Force2(dxtop,Muxtop,Mat,Lxy) < 0 : 
        c = Calculate_Rebar_Force1(dxtop,Muxtop,Mat,Lxy)
    if Calculate_Rebar_Force1(dxtop,Muxtop,Mat,Lxy) > dxtop : 
        c = Calculate_Rebar_Force2(dxtop,Muxtop,Mat,Lxy)
    if Calculate_Rebar_Force2(dxtop,Muxtop,Mat,Lxy) > dxtop : 
        c = Calculate_Rebar_Force1(dxtop,Muxtop,Mat,Lxy)
    if Calculate_Rebar_Force1(dytop,Muytop,Mat,Lxy) < 0 :
        d = Calculate_Rebar_Force2(dytop,Muytop,Mat,Lxy)
    if Calculate_Rebar_Force2(dytop,Muytop,Mat,Lxy) < 0 : 
        d = Calculate_Rebar_Force1(dytop,Muytop,Mat,Lxy)
    if Calculate_Rebar_Force1(dytop,Muytop,Mat,Lxy) > dytop : 
        d = Calculate_Rebar_Force2(dytop,Muytop,Mat,Lxy)
    if Calculate_Rebar_Force2(dytop,Muytop,Mat,Lxy) > dytop : 
        d = Calculate_Rebar_Force1(dytop,Muytop,Mat,Lxy)
    return a, b, c, d

dxbot, dybot, dxtop, dytop = Define_Effective_Depth (Rebar, Muxbot, Muxtop, Muybot, Muytop, dt)
a, b, c, d = Define_Applicable_Rebar_Force (dxbot, dybot, dxtop, dytop, Mat, Lxy)

def Calculate_Required_Rebar_Area(Mat, dt, a, b, c, d) :
    Asxbot = max(0.85*Mat[0]*a*1000/Mat[1], 0.002*1000*(dt+Mat[2])*1000)
    Asybot = max(0.85*Mat[0]*b*1000/Mat[1], 0.002*1000*(dt+Mat[2])*1000)
    Asxtop = max(0.85*Mat[0]*c*1000/Mat[1], 0.002*1000*(dt+Mat[2])*1000)
    Asytop = max(0.85*Mat[0]*d*1000/Mat[1], 0.002*1000*(dt+Mat[2])*1000)
    return Asxbot, Asybot, Asxtop, Asytop

def Calculate_Required_Rebar_Spacing(Asxbot, Asybot, Asxtop, Asytop, Rebar) :
    sy1 = int(1000/(Asxbot/((Rebar[0]/2)**2*3.141592))/50)*50
    sy2 = int(1000/(Asybot/((Rebar[0]/2)**2*3.141592))/50)*50
    sy3 = int(1000/(Asxtop/((Rebar[1]/2)**2*3.141592))/50)*50
    sy4 = int(1000/(Asytop/((Rebar[1]/2)**2*3.141592))/50)*50
    return sy1, sy2, sy3, sy4

Asxbot, Asybot, Asxtop, Asytop = Calculate_Required_Rebar_Area(Mat, dt, a, b, c, d)
sy1, sy2, sy3, sy4 = Calculate_Required_Rebar_Spacing(Asxbot, Asybot, Asxtop, Asytop, Rebar)

print("Muxbot = ", round(Muxbot,0), "kNm (", round(Muxbot,0)/Lxy[1][n-1], " kNm/m)")
print ("Asxbot = ", round(Asxbot,0), "mm2/m")
print ("    = ", round(Asxbot,0) * Lxy[1][n-1], "mm2")
print ("Required Bottom along X axis = D",Rebar[0],"@",sy1)
print ("Provided Bottom along X axis = D",Rebar[0],"@",Rebar[2])
if sy1 > Rebar[1] : 
    F_Capa1 = 'NG'
else : 
    F_Capa1 = 'OK'
print (F_Capa1,'\n')

print("Muybot = ", round(Muybot,0), "kNm (", round(Muybot,0)/Lxy[0][n-1], " kNm/m)")
print ("Asybot = ", round(Asybot,0), "mm2/m")
print ("    = ", round(Asybot,0) * Lxy[0][n-1], "mm2")
print ("Required Bottom along Y axis = D",Rebar[0],"@",sy2)
print ("Provided Bottom along Y axis = D",Rebar[0],"@",Rebar[3])
if sy2 > Rebar[1] : 
    F_Capa2 = 'NG'
else : 
    F_Capa2 = 'OK'
print (F_Capa2,'\n')

print("Muxtop = ", round(Muxtop,0), "kNm (", round(Muxtop,0)/Lxy[1][n-1], " kNm/m)")
print ("Asxtop = ", round(Asxtop,0), "mm2/m")
print ("    = ", round(Asxtop,0) * Lxy[1][n-1], "mm2")
print ("Required Top along X axis = D",Rebar[1],"@",sy3)
print ("Provided Top along X axis = D",Rebar[1],"@",Rebar[4])
if sy3 > Rebar[1] : 
    F_Capa3 = 'NG'
else : 
    F_Capa3 = 'OK'
print (F_Capa3,'\n')

print("Muytop = ", round(Muytop,0), "kNm (", round(Muytop,0)/Lxy[0][n-1], " kNm/m)")
print ("Asytop = ", round(Asytop,0), "mm2/m")
print ("    = ", round(Asytop,0) * Lxy[0][n-1], "mm2")
print ("Required Top along X axis = D",Rebar[1],"@",sy4)
print ("Provided Top along X axis = D",Rebar[1],"@",Rebar[5])
if sy4 > Rebar[1] : 
    F_Capa4 = 'NG'
else : 
    F_Capa4 = 'OK'
print (F_Capa4,'\n')

Muxbot =  1167.0 kNm ( 259.3333333333333  kNm/m)
Asxbot =  1947.0 mm2/m
    =  8761.5 mm2
Required Bottom along X axis = D 22 @ 150
Provided Bottom along X axis = D 22 @ 150
NG 

Muybot =  1173.0 kNm ( 260.6666666666667  kNm/m)
Asybot =  2090.0 mm2/m
    =  9405.0 mm2
Required Bottom along Y axis = D 22 @ 150
Provided Bottom along Y axis = D 22 @ 150
NG 

Muxtop =  167.0 kNm ( 37.111111111111114  kNm/m)
Asxtop =  900.0 mm2/m
    =  4050.0 mm2
Required Top along X axis = D 15 @ 150
Provided Top along X axis = D 15 @ 150
NG 

Muytop =  293.0 kNm ( 65.11111111111111  kNm/m)
Asytop =  900.0 mm2/m
    =  4050.0 mm2
Required Top along X axis = D 15 @ 150
Provided Top along X axis = D 15 @ 150
NG 

